In [1]:
# Import Spark SQL and Spark ML libraries
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.mllib.tree import RandomForest
from pyspark.ml import Pipeline

from pyspark.ml.classification import DecisionTreeClassifier

from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier



In [2]:
IS_SPARK_SUBMIT_CLI = False
if IS_SPARK_SUBMIT_CLI:
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)

In [3]:
if IS_SPARK_SUBMIT_CLI:
    coviddeath = spark.read.csv('UScasestemp1.csv', inferSchema=True, header=True)
else:
    coviddeath = spark.sql("SELECT * FROM uscasestemp1_csv")

In [4]:
data = coviddeath.select("Year","Date","Day", "Temp","Lat","Long","Admin2","Province",((col("Case") > 2).cast("Double").alias("label")))
data = StringIndexer(inputCol='Admin2', outputCol='Admin2'+"_index").fit(data).transform(data)
data = StringIndexer(inputCol='Province', outputCol='Province'+"_index").fit(data).transform(data)
data.show(5)


+----+----+---+----+-----------+------------+-----------+----------+-----+------------+--------------+
Year|Date|Day|Temp| Lat| Long| Admin2| Province|label|Admin2_index|Province_index|
+----+----+---+----+-----------+------------+-----------+----------+-----+------------+--------------+
2020| 22| 1| 61|34.30828379|-118.2282411|Los Angeles|California| 0.0| 3.0| 0.0|
2020| 23| 1| 59|34.30828379|-118.2282411|Los Angeles|California| 0.0| 3.0| 0.0|
2020| 24| 1| 63|34.30828379|-118.2282411|Los Angeles|California| 0.0| 3.0| 0.0|
2020| 25| 1| 64|34.30828379|-118.2282411|Los Angeles|California| 0.0| 3.0| 0.0|
2020| 26| 1| 58|34.30828379|-118.2282411|Los Angeles|California| 0.0| 3.0| 0.0|
+----+----+---+----+-----------+------------+-----------+----------+-----+------------+--------------+
only showing top 5 rows

In [5]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")
train_rows = train.count()
test_rows = test.count()
print ("Training Rows:", train_rows, " Testing Rows:", test_rows)


Training Rows: 305 Testing Rows: 131

In [6]:
from pyspark.ml.classification import RandomForestClassifier
assembler = VectorAssembler(inputCols =["Day","Temp","Lat","Province_index","Admin2_index"],outputCol="normfeatures")
minMax = MinMaxScaler(inputCol = assembler.getOutputCol(), outputCol="nfeatures")
featVect = VectorAssembler(inputCols=["nfeatures"], outputCol="features")
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)
pipeline = Pipeline(stages=[assembler,minMax,featVect,lr])

In [7]:
piplineModel = pipeline.fit(train)
print("Pipeline complete!")


Pipeline complete!

In [8]:
# piplineModel with train data set applies test data set and generate predictions
prediction = piplineModel.transform(test)
predicted = prediction.select("features", "prediction", "trueLabel")
predicted.show(100, truncate=False)

+-----------------------------------------------------------------------------------+----------+---------+
features |prediction|trueLabel|
+-----------------------------------------------------------------------------------+----------+---------+
[0.25,0.24489795918367346,0.48516958731189325,1.0,0.3333333333333333] |0.0 |0.0 |
[0.75,0.5102040816326531,0.48516958731189325,1.0,0.3333333333333333] |1.0 |1.0 |
[0.5,0.5510204081632653,0.48516958731189325,1.0,0.3333333333333333] |1.0 |0.0 |
[0.75,0.30612244897959184,1.0,0.3333333333333333,0.0] |1.0 |1.0 |
[0.75,0.5510204081632653,0.4277898629332098,0.6666666666666666,0.6666666666666666] |1.0 |1.0 |
[1.0,0.8979591836734694,1.0,0.3333333333333333,0.0] |1.0 |1.0 |
[0.5,0.5306122448979592,1.0,0.3333333333333333,0.0] |1.0 |0.0 |
[1.0,0.5714285714285714,1.0,0.3333333333333333,0.0] |1.0 |1.0 |
[0.25,0.673469387755102,0.4277898629332098,0.6666666666666666,0.6666666666666666] |0.0 |0.0 |
[0.5,0.5102040816326531,0.48516958731189325,1.0,0.3333333333333333] |1.0 |0.0 |
[0.5,0.5714285714285714,0.4277898629332098,0.6666666666666666,0.6666666666666666] |1.0 |0.0 |
[0.75,0.3673469387755102,1.0,0.3333333333333333,0.0] |1.0 |1.0 |
[1.0,0.8775510204081632,0.4277898629332098,0.6666666666666666,0.6666666666666666] |1.0 |1.0 |
[1.0,0.8979591836734694,0.48516958731189325,1.0,0.3333333333333333] |1.0 |1.0 |
[0.25,0.08163265306122448,1.0,0.3333333333333333,0.0] |0.0 |0.0 |
[0.25,0.5102040816326531,0.4277898629332098,0.6666666666666666,0.6666666666666666] |0.0 |0.0 |
[0.25,0.6122448979591837,0.0,0.0,1.0] |0.0 |0.0 |
[0.5,0.40816326530612246,0.48516958731189325,1.0,0.3333333333333333] |1.0 |1.0 |
[0.5,0.4489795918367347,0.4277898629332098,0.6666666666666666,0.6666666666666666] |1.0 |0.0 |
[0.5,0.6326530612244898,0.0,0.0,1.0] |1.0 |1.0 |
[1.0,0.6122448979591837,0.0,0.0,1.0] |1.0 |1.0 |
[0.25,0.20408163265306123,0.48516958731189325,1.0,0.3333333333333333] |0.0 |0.0 |
[0.25,0.32653061224489793,0.4277898629332098,0.6666666666666666,0.6666666666666666]|0.0 |0.0 |
[0.5,0.6326530612244898,0.0,0.0,1.0] |1.0 |1.0 |
[0.75,0.7142857142857143,0.48516958731189325,1.0,0.3333333333333333] |1.0 |1.0 |
[1.0,0.8163265306122449,0.0,0.0,1.0] |1.0 |1.0 |
[0.25,0.5102040816326531,0.48516958731189325,1.0,0.3333333333333333] |0.0 |0.0 |
[0.25,0.6122448979591837,0.0,0.0,1.0] |0.0 |0.0 |
[0.5,0.3469387755102041,0.48516958731189325,1.0,0.3333333333333333] |1.0 |1.0 |
[0.75,0.6122448979591837,0.0,0.0,1.0] |1.0 |1.0 |
[0.75,0.6326530612244898,1.0,0.3333333333333333,0.0] |1.0 |1.0 |
[0.75,0.7551020408163265,0.48516958731189325,1.0,0.3333333333333333] |1.0 |1.0 |
[0.25,0.6122448979591837,0.0,0.0,1.0] |0.0 |0.0 |
[0.5,0.5714285714285714,0.48516958731189325,1.0,0.3333333333333333] |1.0 |1.0 |
[0.5,0.5918367346938775,0.4277898629332098,0.6666666666666666,0.6666666666666666] |1.0 |0.0 |
[0.5,0.673469387755102,0.0,0.0,1.0] |1.0 |1.0 |
[1.0,0.5510204081632653,0.4277898629332098,0.6666666666666666,0.6666666666666666] |1.0 |1.0 |
[1.0,0.9183673469387755,0.0,0.0,1.0] |1.0 |1.0 |
[1.0,0.9183673469387755,1.0,0.3333333333333333,0.0] |1.0 |1.0 |
[0.5,0.8163265306122449,0.4277898629332098,0.6666666666666666,0.6666666666666666] |1.0 |0.0 |
[0.5,0.8367346938775511,0.48516958731189325,1.0,0.3333333333333333] |1.0 |1.0 |
[0.75,0.5714285714285714,0.48516958731189325,1.0,0.3333333333333333] |1.0 |1.0 |
[0.75,0.6122448979591837,0.0,0.0,1.0] |1.0 |1.0 |
[0.75,0.7346938775510204,0.4277898629332098,0.6666666666666666,0.6666666666666666] |1.0 |1.0 |
[1.0,0.8979591836734694,1.0,0.3333333333333333,0.0] |1.0 |1.0 |
[0.5,0.6326530612244898,0.0,0.0,1.0] |1.0 |1.0 |
[0.75,0.6122448979591837,0.0,0.0,1.0] |1.0 |1.0 |
[0.75,0.7959183673469388,1.0,0.3333333333333333,0.0] |1.0 |1.0 |
[0.25,0.7142857142857143,0.0,0.0,1.0] |0.0 |0.0 |
[0.75,0.6530612244897959,0.48516958731189325,1.0,0.3333333333333333] |1.0 |1.0 |
[0.25,0.3877551020408163,1.0,0.3333333333333333,0.0] |0.0 |0.0 |
[0.75,0.4897959183673469,1.0,0.3333333333333333,0.0] |1.0 |1.0 |
[0.75,0.6530612244897959,0.0,0.0,1.0

In [9]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluation = MulticlassClassificationEvaluator(
    labelCol="trueLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluation.evaluate(prediction)
print("Accuracy of Logistic Regression is: ",accuracy)

Accuracy of Logistic Regression is: 0.916030534351145

In [10]:
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.0839695